# Tissue Segmentation with Voyager

Based on the following tutorials:
* https://pachterlab.github.io/voyager/articles/visium_10x.html
* https://pachterlab.github.io/voyager/articles/vig1_visium_basic.html
* https://pachterlab.github.io/voyager/articles/vig2_visium.html
* https://pachterlab.github.io/voyager/articles/visium_10x_spatial.html
* https://pachterlab.github.io/voyager/articles/multispati.html

In [ ]:
library(dplyr)
library(Voyager)
library(SpatialExperiment)
library(SpatialFeatureExperiment)
library(SingleCellExperiment)
library(ggplot2)
library(scater)
library(rlang)
library(scran)
library(scuttle)
library(terra)
library(sf)
library(rmapshaper)
library(scran)
library(stringr)
library(EBImage)
library(patchwork)
library(bluster)
library(rjson)
theme_set(theme_bw())

In [ ]:
# Layout
custom_theme <- function() {
  theme_bw() +
    theme(
      legend.text = element_text(size = 14),
      legend.title = element_text(size = 16, face = "bold"),
      axis.text = element_text(size = 12),
      axis.title = element_text(size = 14, face = "bold"),
      legend.position = "right",
      legend.box.just = "right"
    )
}
options(repr.plot.width = 10, repr.plot.height = 8)

In [ ]:
data_dir <- R.utils::getAbsolutePath('../../data')
input_dir <- glue::glue("{data_dir}/Visium_Skin_A2/outs")

## Visium Files

### Scale Factors

The scalefactors_json.json file contains image metadata:
* **tissue_hires_scalef** and **tissue_lowres_scalef** are the ratio of the size of the high resolution (but not full resolution) and low resolution H&E image to the full resolution image.
* **fiducial_diameter_fullres** is the diameter of each fiducial spot used to align the spots to the H&E image in pixels in the full resolution image.
* **spot_diameter_fullres** is the diameter of each Visium spot in the full resolution H&E image in pixels. 

In [ ]:
scale_factors <- fromJSON(file = glue::glue("{input_dir}/spatial/scalefactors_json.json"))
scale_factors

### Tissue Metadata

The tissue_positions_list.csv file contains information about each spot/barcode:
* **in_tissue** indicates whether each spot is in tissue (in_tissue, 1 means yes and 0 means no) as automatically detected by 
Space Ranger or manually annotated in the Loupe browser.
* **array_row** and **array_col** are the coordinates on the matrix of spots,
* **pxl_row_in_fullres** and **pxl_col_in_fullres** are the coordinates of the spots in the full resolution 
image.

In [ ]:
head(read.csv(glue::glue("{input_dir}/spatial/tissue_positions.csv")))

## Read Visium Data

In [ ]:
raw_sfe <- SpatialFeatureExperiment::read10xVisiumSFE(dirs = input_dir, samples = ".", type = "sparse", data = "raw")

In [ ]:
Voyager::plotImage(raw_sfe)

## Read Hi-res Image

In [ ]:
img <- readImage(glue::glue("{input_dir}/spatial/tissue_hires_image.png"))
EBImage::display(img)

In [ ]:
img2 <- img
EBImage::colorMode(img2) <- EBImage::Grayscale
EBImage::display(img2, all = TRUE)
(EBImage::hist(img) + custom_theme())

In [ ]:
mask <- img2[,,3] < 0.90
kern <- EBImage::makeBrush(3, shape='disc')
mask_open <- EBImage::opening(mask, kern)
mask_close <- EBImage::closing(mask_open, kern)
EBImage::display(mask_open)

In [ ]:
mask_label <- EBImage::bwlabel(mask_close)
fts <- EBImage::computeFeatures.shape(mask_label)
max_ind <- terra::which.max(fts[,"s.area"])
inds <- which(as.array(mask_label) == max_ind, arr.ind = TRUE)
row_inds <- c(seq_len(min(inds[,1])-1), seq(max(inds[,1])+1, nrow(mask_label), by = 1))
col_inds <- c(seq_len(min(inds[,2])-1), seq(max(inds[,2])+1, nrow(mask_label), by = 1))
# mask_label[row_inds, ] <- 0
# mask_label[,col_inds] <- 0
fts2 <- fts[unique(as.vector(mask_label))[-1],]
fts2 <- fts2[order(fts2[,"s.area"], decreasing = TRUE),]
mask_label[mask_label %in% c(797, as.numeric(rownames(fts2)[fts2[,1] < 100]))] <- 0
mask_label <- EBImage::fillHull(mask_label)
EBImage::display(mask_label)

In [ ]:
plot(fts2[,1][-1], type = "l", ylab = "Area")

In [ ]:
head(fts2, 10)

In [ ]:
mask_label[mask_label %in% c(797, as.numeric(rownames(fts2)[fts2[,1] < 100]))] <- 0

In [ ]:
display(paintObjects(mask_label, img, col=c("red", "yellow"), opac=c(1, 0.3)))

In [ ]:
raster2polygon <- function(seg, keep = 0.2) {
    seg <- flip(seg)
    r <- terra::rast(as.array(seg), extent = ext(0, nrow(seg), 0, ncol(seg))) |> trans()
    r[r < 1] <- NA
    contours <- st_as_sf(as.polygons(r, dissolve = TRUE))
    simplified <- ms_simplify(contours, keep = keep)
    return(list(full = contours, simplified = simplified))
}

In [ ]:
tb <- raster2polygon(mask_label)
tb

In [ ]:
scale_factors <- fromJSON(file = glue::glue("{input_dir}/spatial/scalefactors_json.json"))
tb$simplified$geometry <- tb$simplified$geometry / scale_factors$tissue_hires_scalef

In [ ]:
Voyager::plotImage(raw_sfe)

In [ ]:
is_mt <- str_detect(rowData(raw_sfe)$symbol, "^MT-")
sum(is_mt)

In [ ]:
segmented_sfe <- scuttle::addPerCellQCMetrics(raw_sfe, subsets = list(mito = is_mt))
colData(segmented_sfe)$nCounts <- colSums(counts(segmented_sfe))

In [ ]:
SpatialFeatureExperiment::tissueBoundary(segmented_sfe) <- tb$simplified

In [ ]:
Voyager::plotSpatialFeature(segmented_sfe, "sum", annotGeometryName = "tissueBoundary", 
                   annot_fixed = list(fill = NA, color = "black"),
                   image_id = "lowres") + custom_theme()

In [ ]:
segmented_sfe <- SpatialFeatureExperiment::transpose(segmented_sfe)

In [ ]:
Voyager::plotSpatialFeature(segmented_sfe, "sum", annotGeometryName = "tissueBoundary", 
                   annot_fixed = list(fill = NA, color = "black"),
                   image_id = "lowres")

In [ ]:
segmented_sfe$int_tissue <- SpatialFeatureExperiment::annotPred(segmented_sfe, colGeometryName = "spotPoly", 
                            annotGeometryName = "tissueBoundary",
                            pred = st_intersects)
segmented_sfe$cov_tissue <- SpatialFeatureExperiment::annotPred(segmented_sfe, colGeometryName = "spotPoly", 
                            annotGeometryName = "tissueBoundary",
                            pred = st_covered_by)

In [ ]:
segmented_sfe$diff_sr <- 
    dplyr::case_when(
        segmented_sfe$in_tissue == segmented_sfe$int_tissue ~ "same",
        segmented_sfe$in_tissue & !segmented_sfe$int_tissue ~ "Space Ranger",
        segmented_sfe$int_tissue & !segmented_sfe$in_tissue ~ "segmentation"
    ) |> 
    factor(levels = c("Space Ranger", "same", "segmentation"))
Voyager::plotSpatialFeature(
    segmented_sfe, "diff_sr", 
    annotGeometryName = "tissueBoundary", 
    annot_fixed = list(fill = NA, size = 0.5, color = "black")) +
    scale_fill_brewer(type = "div", palette = 4) + custom_theme()

In [ ]:
segmented_sfe$diff_int_cov <- segmented_sfe$int_tissue != segmented_sfe$cov_tissue
Voyager::plotSpatialFeature(
    segmented_sfe, 
    "diff_int_cov", 
    annotGeometryName = "tissueBoundary", 
    annot_fixed = list(fill = NA, size = 0.5, color = "black")) + custom_theme()

In [ ]:
# colData(sfe)$nCounts <- colSums(counts(sfe))
violin <- plotColData(segmented_sfe, "sum", x = "in_tissue", colour_by = "in_tissue") +
    theme(legend.position = "top")
spatial <- plotSpatialFeature(segmented_sfe, "sum", colGeometryName = "spotPoly",
                              annotGeometryName = "tissueBoundary", 
                              image = "lowres", maxcell = 5e4,
                              annot_fixed = list(fill = NA, color = "black")) +
    custom_theme()
violin + spatial

In [ ]:
spot_ints <- annotOp(segmented_sfe, colGeometryName = "spotPoly", annotGeometryName = "tissueBoundary", op = st_intersection)
segmented_sfe$pct_tissue <- st_area(spot_ints) / st_area(spotPoly(segmented_sfe)) * 100
sfe_tissue <- segmented_sfe[, segmented_sfe$int_tissue]
scater::plotColData(sfe_tissue, x = "pct_tissue", y = "sum", color_by = "diff_int_cov") + custom_theme()

In [ ]:
data_dir <- R.utils::getAbsolutePath('../../data')
saveRDS(segmented_sfe, glue::glue("{data_dir}/Visium_Skin_A2.rds"))